# ISWC 2019 Cell-Entity Annotation (CEA) Challenge
https://www.aicrowd.com/challenges/iswc-2019-cell-entity-annotation-cea-challenge

## Opis zadania

Zadanie polega na anotacji zawartości poszczególnych komórek z jednostkami dbpedii o prefiksie “http://dbpedia.org/resource/”. Jedna komórka może być powiązana z tylko jednym zasobem. 
Końcowym wynikiem powinien być plik zawierający informacje o id tabeli oraz wierszu i kolumnie w jakim znajduje się komórka wraz z anotacją. Np.
“9206866_1_8114610355671172497”,”0”,”121”,”http://dbpedia.org/resource/Norway”

## Przygotowanie środowiska

Instalacja bibliotek wymaganych do prawidłowego uruchomienia notatnika.

In [1]:
%pip install pandas==1.3.4
%pip install requests==2.26.0
%pip install python-Levenshtein==0.12.2
%pip install spotlight==2.3.2

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'e:\pytong\CEA\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'e:\pytong\CEA\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'e:\pytong\CEA\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'e:\pytong\CEA\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


### Ekstrakcja danych

Na samym początku należy wyodrębnić dane do pracy. Do zadania został dołączony zbiór plików csv, gdzie jeden plik odpowiada jednej tabeli, a jego nazwa określa id tej tabeli. Pliki te nie zawierają wiersza nagłówkowego. 
Dodatkowo dostarczony jest plik w którym znajdują się informacje o docelowych komórkach do anotacji (id tabeli, wiersz oraz kolumna komórki).
Ilość komórek do anotacji wynosi łącznie 8418.

Za pomocą poniższego kodu odczytywana jest zawartość plików (tabel) oraz wyszukiwane docelowe wartości komórek do anotacji.

In [3]:
import pandas as pd

# Load data
df_target = pd.read_csv("data/Round 1/targets/CEA_Round1_Targets.csv", header=None)
df_target.columns = ["Table_id", "Column_id", "Row_id"]

# Read all targets and find them in data tables
df_target = df_target.assign(text=None)
for index, row in df_target.iterrows():
    try:
        table_id = row['Table_id']
        column_id = int(row['Column_id'])
        row_id = int(row['Row_id'])
        table = pd.read_csv(f'data/Round 1/tables/{table_id}.csv', header=None)
        text = table.loc[row_id][column_id]
        df_target.loc[index, 'text'] = text
    except Exception as exc:
        print(f"EXCEPTION {exc}")

In [4]:
df_target.head()

,Table_id,Column_id,Row_id,text
0,50245608_0_871275842592178099,0,154,Pearl Harbor
1,39107734_2_2329160387535788734,1,32,Minneapolis-Saint Paul International Airport
2,22864497_0_8632623712684511496,0,227,Need for Speed: Carbon
3,66009064_0_9148652238372261251,0,15,Alex Rodriguez
4,21362676_0_6854186738074119688,1,75,King Kong


### Czyszczenie danych

Następnie przeszliśmy do wyczyszczenia uzyskanych z komórek tekstów ze znaków specjalnych, które mogłyby w kolejnych krokach utrudniać zadanie. Dostosowanie zbioru znaków do usunięcia wymagało kilku iteracji i porównania skuteczności zachowania dalszych kroków. W ten sposób zauważyliśmy na przykład, że lepsze wyniki są uzyskiwanie kiedy w tekście pozostawiany jest znak myślnika (“-”) niż w przypadku gdy jest on pomijany.

Poniższa funkcja przyjmuje tekst i zwraca jego wersję bez znaków uznanych za utrudniające anotacje.

In [5]:
chars = '!"#$%&\'()*+,./:;<=>?@[\\]^`{|}~'

def clear_entity(cell_entity: str) -> str:
    if not isinstance(cell_entity, str):
        return cell_entity
    clear_cell = ""
    for ch in cell_entity:
        if ch not in chars:
            clear_cell += ch
    return clear_cell.strip()
    
df_target['text'] = df_target['text'].map(clear_entity)

In [6]:
df_target.head()

,Table_id,Column_id,Row_id,text
0,50245608_0_871275842592178099,0,154,Pearl Harbor
1,39107734_2_2329160387535788734,1,32,Minneapolis-Saint Paul International Airport
2,22864497_0_8632623712684511496,0,227,Need for Speed Carbon
3,66009064_0_9148652238372261251,0,15,Alex Rodriguez
4,21362676_0_6854186738074119688,1,75,King Kong


## Anotacja

Po przygotowaniu danych mogliśmy przejść do docelowego zadania - anotacji. Ten etap rozpoczęliśmy od zapoznania się z dokumentami stworzonymi przez uczestników konkursu, którzy uzyskali najlepsze wyniki w rankingu. Przekrój stworzonych rozwiązań był bardzo różnorodny, więc ostatecznie zadecydowaliśmy się wykorzystać kilka sposobów, połączyć je ze sobą i porównać uzyskane wyniki. Ostatecznie zadecydowaliśmy się wykorzystać 3 sposoby wyznaczania połączeń z dbpedią. Naszym celem było zebranie kandydatów wyłonionych za pomocą każdego z nich i ostatecznie na samym końcu wyłonienie najlepiej pasującej strony.

### Samodzielne tworzenie adresu URL

Pierwsze, zdecydowanie najprostsze rozwiązanie, zakłada wykorzystanie wzoru zgodnie z którym są tworzone adresy url dla zasobów dbpedii. Każdy z takich zasobów rozpoczyna się prefiksem “http://dbpedia.org/resource/” a następny element odpowiada nazwie zasobu. Wykorzystując tą wiedze można w tekście do anotacji zamienić znak spacji na znak podkreślenia (”_”), a następnię dołączenie go do znanego prefiksu. Dzięki temu tworzony jest adres odpowiadający strukturą zasobom znajdującym się na stronie. W tym momencie można przejść do odpytania samodzielnie stworzonego adresu i jeśli w odpowiedzi otrzymany jest prawidłowy kod statusu, to zasób o danym adresie istnieje. Można założyć z dużym prawdopodobieństwem że jest to szukany wynik. 

In [7]:
import requests

def check_url(enity_value: str):
    try:
        parsed_data = enity_value.replace(" ", "_")
        url = f'http://dbpedia.org/resource/{parsed_data}'
        resp = requests.get(url, headers={'Connection': 'close'})
        assert resp.status_code == 200
        return url
    except AssertionError:
        # Invalid url
        return None

    except Exception as exc:
        print(f"EXC {exc}")
        return None

In [8]:
check_url("Buddy Rosar")

'http://dbpedia.org/resource/Buddy_Rosar'

### DBpedia Lookup Service - Auto-Complete API 

DBpedia Lookup Service jest to usługa wyszukiwania zwracająca jednostki DBpedii. Może zostać skonfigurowana na łączenie słów kluczowych z identyfikatorami zasobów. Każdy tak uzyskany wynik posiada liczbę wyznaczaną na podstawie dopasowań etykiet oraz innych czynników. Możliwe jest ograniczenie zwracanych wyników o wymaganie miimalnej jej wartości. Pozwala to na odrzucenie wyników które mają małe prawdopodobieństwo na okazanie się docelowo szukanymi.

W trakcie wykonywania zadania napotkaliśmy problem, gdy serwer hostowany przez dbpedie przestał działać i każde zapytanie zwracało kod statusu 500. W takiej sytuacji wykorzystaliśmy kod udostępniony na github.com oraz już zbudowane obrazy dockerowe. Z ich pomocą uruchomiliśmy aplikacje lokalnie. Pozwoliło nam to na dalsze testy i prace. Wykorzystane wersje różnią się minimalnie między sobą dlatego poniżej znajdują się 2 rozwiązania pozyskania wyników w zależności od wykorzystania serwisu udostępnianego publicznie lub samodzielnego hostowania lokalnego.

In [14]:
import requests
import xmltodict
import urllib3
requests.urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def dbpedia_lookup(enity_value: str, max_hits=20):
    url = f'https://lookup.dbpedia.org/api/search/KeywordSearch?MaxHits={max_hits}&QueryString="{enity_value}"'
    # url = f'http://localhost:9274/lookup-application/api/search?query={enity_value}&maxResults={max_hits}'
    try:
        resp = requests.get(url, headers={'Connection': 'close'}, verify=False)
        if resp.status_code != 200:
            print('Service Unavailable')
            return []
        result_tree = xmltodict.parse(resp.content.decode('utf-8'))
        if result_tree['ArrayOfResults'] is None or 'Result' not in result_tree['ArrayOfResults']:
            print(enity_value)
            return []
        result = result_tree['ArrayOfResults']['Result']
        if isinstance(result, list):
            return [r['URI'] for r in result]

        return [result['URI']]
    except Exception as exc:
        print(f"EXC {exc}, {enity_value}")


In [15]:
dbpedia_lookup('Lappet-faced vulture')

['http://dbpedia.org/resource/Lappet-faced_vulture']

### DBPedia Spotlight API

DBpedia Spotlight jest to narzędzie pozwalające na automatyczne dodawanie adnotacji do wzmianek zasobów DBpedii w tekście. Zapewnia rozwiązanie do łączenia nieustrukturyzowanych źródeł informacji z zasobami DBpedii. Zdecydowanie bardziej złożone rozwiązanie w porównaniu do poprzednich sposobów. Wykorzystuje 4-stopniowy proces rozpoznawania danych, bazując na kontekście w jakim występuje tekst. Sposób wykorzystania w naszym przypadku nie jest więc optymalny. Na pewno lepiej  odpowiadałby na potrzebe anotacji dłuższych tekstów (kolumn lub nawet całych tabel).


Istnieje możliwość analogicznego uruchomienia systemu lokalnie jak w przypadku DBpedia Lookup Service, ale nie wystąpiły żadne problemy z dostępnym api, więc do tego zadania zdecydowaliśmy się je wykorzystać. 

In [12]:
import requests


def spotlight_lookup(text, lang='en', confidence=0.01):
    url = f"https://api.dbpedia-spotlight.org/{lang}/annotate"
    params = {'text': text, 'confidence':confidence}
    headers = {'accept': 'application/json'}

    try:
        response = requests.request("GET", url, headers=headers, params=params)
        results = response.json()['Resources']
        matches = []
        for result in results:
            uri = result['@URI']         
            matches.append(uri.replace('/page/', '/resource/'))
    except Exception as e:
        print(f'[SPOTLIGHT] Something went wrong with text: {text}. Returning nothing')
        return []

    return matches


In [13]:
spotlight_lookup("Lappet-faced vulture")

['http://dbpedia.org/resource/Lappet-faced_vulture']

## Wyniki

### Odległość Levenshtein’a

Do wytypowania najlepszego wyniku zdecydowaliśmy się na wykorzystanie odległości Levenshtein’a. Oznacza ona najmniejszą liczbę działań prostych, przeprowadzająca jeden napis w drugi.

In [16]:
import Levenshtein

def find_best_match(urls, cell_value):
    min_distance = 9999
    value = ''
    for url in urls:
        label = url.split('resource')[1][1:].replace('_', ' ')
        levenshtein_distance = Levenshtein.distance(cell_value.lower(), label.lower())
        if min_distance > levenshtein_distance:
            value = url
            min_distance = levenshtein_distance

    return value, min_distance


Ostateczne wyznaczenie wyników za pomocą wszystkich sposobów i wyłonienie potencjalnych kandydatów ostatecznego wyniku. 

In [17]:
import pandas as pd  


def save_results(cell, index, urls, dataframe, filename):
        # Levenshtein
        best_match, distance = find_best_match(urls, cell)

        # save_results
        dataframe.loc[index, 'annotation'] = best_match
        dataframe.loc[index, 'candidates'] = str(urls)

        print(cell, best_match, distance)
        dataframe.to_csv(f'{filename}_results.csv', sep=',', index=False, )


def annotate(cleared_data: pd.DataFrame):
    result_df = cleared_data.assign(annotation=None, candidates=None)
    
    url_check_results = result_df.copy()
    dbpedia_lookup_results_df = result_df.copy()
    spotlight_lookup_results_df = result_df.copy()
    
    for index, row in result_df.iterrows():
        cell = row['text']
        if not isinstance(cell, str):
            continue
        
        result_candidates = []
        
        # CHECK
        url = check_url(cell)
        if url is not None:
            result_candidates.append(url)
            save_results(cell, index, [url], url_check_results, 'check_url')
        
        # DBPEDIA
        dbpedia_lookup_urls = dbpedia_lookup(cell, 10)
        for url in dbpedia_lookup_urls:
            result_candidates.append(url)
        save_results(cell, index, dbpedia_lookup_urls, dbpedia_lookup_results_df, 'dbpedia_lookup')
        
        # SPOTLIGHT
        spotlight_lookup_urls = spotlight_lookup(cell)
        for url in spotlight_lookup_urls:
            if url not in result_candidates:
                result_candidates.append(url)
        save_results(cell, index, spotlight_lookup_urls, spotlight_lookup_results_df, 'spotlight_lookup')

        save_results(cell, index, result_candidates, result_df, 'all')


In [ ]:
annotate(df_target)

Pearl Harbor http://dbpedia.org/resource/Pearl_Harbor 0
Pearl Harbor http://dbpedia.org/resource/Pearl_Harbor 0
Pearl Harbor http://dbpedia.org/resource/Pearl_Harbor 0
Pearl Harbor http://dbpedia.org/resource/Pearl_Harbor 0
Minneapolis-Saint Paul International Airport http://dbpedia.org/resource/Minneapolis-Saint_Paul_International_Airport 0
Minneapolis-Saint Paul International Airport http://dbpedia.org/resource/Minneapolis–Saint_Paul_International_Airport 1
Minneapolis-Saint Paul International Airport http://dbpedia.org/resource/Minneapolis–Saint_Paul_International_Airport 1
Minneapolis-Saint Paul International Airport http://dbpedia.org/resource/Minneapolis-Saint_Paul_International_Airport 0
Need for Speed Carbon http://dbpedia.org/resource/Need_for_Speed_Carbon 0
Need for Speed Carbon http://dbpedia.org/resource/Need_for_Speed:_Carbon 1
Need for Speed Carbon http://dbpedia.org/resource/Carbon 15
Need for Speed Carbon http://dbpedia.org/resource/Need_for_Speed_Carbon 0
Alex Rodrigue

## Porównanie uzyskanych rezultatów

In [18]:
expected_results = pd.read_csv("data/Round 1/gt/CEA_Round1_gt.csv", 
                                header=None, 
                                names=['Table_id', 'Column_id', 'Row_id', 'Expected_annotations'])

def compare_results(results: pd.DataFrame, expected_results: pd.DataFrame):
    # Precision = (# correctly annotated cells) / (# annotated cells)
    # Recall = (# correctly annotated cells) / (# target cells)    
    correctly_annotated_cells = 0
    annotated_cells = len(results.index)
    target_cells = len(expected_results.index)

    final_df = pd.merge(results, expected_results,
                        how="left", on=['Table_id', 'Column_id', 'Row_id'])

    for _, row in final_df.iterrows():
        if row['text'] is None or row['annotation'] is None:
            annotated_cells -=1
            continue
        if str(row['annotation']) in row['Expected_annotations']:
            correctly_annotated_cells += 1
    
    precision = correctly_annotated_cells/annotated_cells
    recall = correctly_annotated_cells/target_cells

    return final_df, precision, recall


In [22]:
check_url_results = pd.read_csv("check_url_results.csv", header=0)
_, precision, recall = compare_results(check_url_results, expected_results)
print(f'Samodzielne tworzenie url: precyzja {round(precision,2)}')
# print(f'check_url_results recall {round(recall,2)}')


dbpedia_lookup_results = pd.read_csv("dbpedia_lookup_results.csv", header=0)
_, precision, recall = compare_results(dbpedia_lookup_results, expected_results)
print(f'DBpedia Lookup Service: precyzja {round(precision,2)}')
# print(f'DBpedia Lookup Service recall {round(recall,2)}')


spotlight_lookup_results = pd.read_csv("spotlight_lookup_results.csv", header=0)
_, precision, recall = compare_results(spotlight_lookup_results, expected_results)
print(f'DBpedia Spotlight: precyzja {round(precision,2)}')
# print(f'DBpedia Spotlight recall {round(recall,2)}')


all_results = pd.read_csv("all_results.csv", header=0)
_, precision, recall = compare_results(all_results, expected_results)
print(f'Ostateczny wynik: precyzja {round(precision,2)}')
# print(f'Ostateczny wynik: recall {round(recall,2)}')


Samodzielne tworzenie url: precyzja 0.79
DBpedia Lookup Service: precyzja 0.8
DBpedia Spotlight: precyzja 0.68
Ostateczny wynik: precyzja 0.89


## Podsumowanie

Wyniki okazały się zgodne z oczekiwaniami. Najmniej skutecznym samodzielnym sposobem okazało się wykorzystanie narzędzia DBpedia Spotlight API. Tak jak opisywane wcześniej jest to narzędzie najlepiej radzące sobie w przypadku dłuższych tekstów, a nie jak w przypadku tego zadania pojedynczych słów/ nazw. Ciekawym faktem jest iż wykorzystanie sposobu ręcznego tworzenia adresów ma podobną skuteczność jak wykorzystanie narzędzia DBpedia Lookup. Ostateczny wynik, łączący wszystkie metody uzyskał precyzje ~0.89 co wydaje się zadowalającym rezultatem.

## Propozycje możliwego dalszego rozwoju

Na podstawie analizy poprawnie i niepoprawnie rozpoznanych danych możliwe byłoby dalsze usprawnianie całego systemu. Trzeci sposób anotacji (DBpedia Spootligh) zdecydowanie zyskałby w momencie w którym dołączona do zapytania byłaby chociaż kategoria do jakiej się odnosi. Narzędzie pozwala na bardzo rozbudowane zapytania, które w tym przypadku pojedynczych krótkich tekstów ciężko było wykorzystać.  

Np. niektóre komórki zawierały pierwszą literę imienia oraz nazwisko. W przypadku poświęcenia dodatkowego czasu na wykrycie takiej specyficznej sytuacji, możliwe byłoby dopisanie kategori do zapytania lub wyszukanie encji za pomocą samego nazwiska.

# Bibliografia

* https://www.dbpedia.org/resources/lookup/
* https://github.com/dbpedia/dbpedia-lookup
* https://www.dbpedia-spotlight.org/
* http://www.cs.ox.ac.uk/isg/challenges/sem-tab/
* http://ceur-ws.org/Vol-2553/
* http://ceur-ws.org/Vol-2775/paper9.pdf
* http://ceur-ws.org/Vol-2553/paper5.pdf
